 ## Things to try to increase LB score

 1. Change way in which buffer region is constucted (Cross, Square, Ball)

 1. Change submission csv method to transpose rows and cols

 ### Submitting first 200 000, we have

 * 0.025533852 accuracy with only zeros.

 * 0.00043453 accuracy with only ones.

 * ie, 0.00043453 / (0.00043453 + 0.025533852) = 0.01673304097 ~= **1.67 %** rate of ones



 ....We should maybe consider that when setting threshold.

 ### Estimated time for getting **25m rows**:

 * One forward pass gives 40 000 rows

 * One forward pass takes approx. 0.5 sec

 * 25m / 40 000 = 625

 * 625 * 0.5 ~= 300 sec = **5 min**

 # Inference Notebook



 This notebook performs inference on test AOIs using the trained model. It reads the AOIs from shapefiles, processes each AOI through the model, and aggregates the results into final prediction tensors. The predictions are then converted into submission CSV files.



 The methods from `inference_utils.py` and `train_utils.py` are imported and used.

**TODO** change model to do 224 -> 224 or 224 -> 112. Otherwise, rounding errors m

 ## Imports and Setup

In [1]:
import os
import sys
import numpy as np
import torch
import random
from tqdm.notebook import tqdm

# Add the src directory to the sys.path
sys.path.append(os.path.abspath('..'))

# Import functions and constants from inference_utils and train_utils
from secret_runway_detection.inference_utils import (
    has_strip_tensors_to_submission_csv,
)

from secret_runway_detection.train_utils import (
    add_buffer_to_label,
)

from secret_runway_detection.model import get_model

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


 ## Configuration Parameters

In [2]:
DEBUG = True

ADD_BUFFER = True
BUFFER_TYPE = 'cross'

# Tile and AOI parameters
TILE_SIDE_LEN = 10.0  # in meters
# AOI_HEIGHT = 15270.0  # in meters
# AOI_WIDTH = 15410.0   # in meters

# # ROWS_COUNT = 1527  # Number of tile rows
# # COLUMNS_COUNT = 1541  # Number of tile columns

# assert (TILE_SIDE_LEN == AOI_HEIGHT / ROWS_COUNT) and (TILE_SIDE_LEN == AOI_WIDTH / COLUMNS_COUNT)

# Model input and output dimensions
INPUT_IMAGE_SIDE_LEN_PX = 224  # in pixels
TILES_PER_INPUT_AREA_LEN = 224  # Number of tiles per side in one input area

# Number of input areas to cover the AOI
INPUT_AREAS_VERTICALLY = 10
INPUT_AREAS_HORIZONTALLY = 10

# Threshold for converting model outputs to binary predictions
THRESHOLD = 0.5  # Adjust based on validation performance

# Path to the trained model checkpoint
RUN_PATH = 'esedx12/secret-runway-detection/f1e9mc58'

# Path to save the submission CSVs
SUBMISSION_CSV_DIR = 'submission_csvs'
os.makedirs(SUBMISSION_CSV_DIR, exist_ok=True)


 ## Load the Trained Model

In [3]:
# Load the W&B run
import wandb


train_run = wandb.Api().run(RUN_PATH)
train_run.name

'wild-forest-84'

In [5]:
from pathlib import Path
from more_itertools import one

# Fetch the model artifact from the W&B run
artifacts = train_run.logged_artifacts()
model_artifacts = [a for a in artifacts if a.type == 'model']
artifact = one(model_artifacts)
state_dict_dir = artifact.download(root='../artifacts/')
state_dict_dir = Path(state_dict_dir)
state_dict_path = state_dict_dir / f'{train_run.name}.pth'
state_dict_path

# %%
model_type = train_run.config['model_type']
print(f"Model type: {model_type}")

# %%
# Now load the type backbone_config artifact the same way you loaded the model and put it in a temp directory
backbone_config_artifacts = [a for a in artifacts if a.type == 'backbone_config']
backbone_config_artifact = one(backbone_config_artifacts)
backbone_config_dir = backbone_config_artifact.download(root='../artifacts/')
backbone_config_dir = Path(backbone_config_dir)
backbone_config_path = backbone_config_dir / f'{train_run.name}.yaml'
backbone_config_path


# %%
list(artifacts)

for artifact in artifacts:
    print(artifact.type, artifact.name)

# %%
# Then load the model like this
# model = get_model(config['model_type'], BACKBONE_CFG_PATH, backbone_weights_path, output_size=config['resolution']).to(device)
sys.path.append(os.path.abspath('../GFM'))

from secret_runway_detection.model import get_model

BACKBONE_CFG_PATH = '../configs/gfm_config.yaml'
backbone_weights_path = '../simmim_pretrain/gfm.pth'


model = get_model(train_run.config['model_type'], BACKBONE_CFG_PATH, backbone_weights_path, output_size=train_run.config['resolution']).to(device)

# %%
# Load the Model from WandB, which we saved as state dict
# Load the Model from WandB, which we saved as state dict
state_dict = torch.load(state_dict_path, map_location=device)
incompatible_keys = model.load_state_dict(state_dict, strict=False)
print(f"Incompatible keys: {incompatible_keys}")
model.eval()

# %%
best_threshold = train_run.summary['best_threshold']



wandb: Downloading large artifact model:v29, 406.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.0


Model type: midfeat


wandb:   1 of 1 files downloaded.  


backbone_config gfm_config:v1
model model:v29
=> merge config from ../configs/gfm_config.yaml


/home/emil/Desktop/secret-runway-detection/.venv/lib/python3.12/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]



Loading pretrained weights...

Missing keys when loading pretrained weights:
  - head.weight
  - head.bias

Unexpected keys when loading pretrained weights:
  - mask_token

Pretrained weights loaded with some missing/unexpected keys.
Incompatible keys: _IncompatibleKeys(missing_keys=['decode_head.resblocks.1.conv1.weight', 'decode_head.resblocks.1.conv1.bias', 'decode_head.resblocks.1.bn1.weight', 'decode_head.resblocks.1.bn1.bias', 'decode_head.resblocks.1.bn1.running_mean', 'decode_head.resblocks.1.bn1.running_var', 'decode_head.resblocks.1.conv2.weight', 'decode_head.resblocks.1.conv2.bias', 'decode_head.resblocks.1.bn2.weight', 'decode_head.resblocks.1.bn2.bias', 'decode_head.resblocks.1.bn2.running_mean', 'decode_head.resblocks.1.bn2.running_var', 'decode_head.resblocks.2.conv1.weight', 'decode_head.resblocks.2.conv1.bias', 'decode_head.resblocks.2.bn1.weight', 'decode_head.resblocks.2.bn1.bias', 'decode_head.resblocks.2.bn1.running_mean', 'decode_head.resblocks.2.bn1.running_var

## Load images of AOIs

In [7]:
# %% [markdown]
# ## Load AOI Mosaics Based on Shapefile Names and Check for Missing Mosaics

# %%
import os
import rasterio
import numpy as np

# Directory containing AOI shapefiles
AOI_SHAPEFILES_DIR = '../shp_test_AOIs/shp'  # Adjust the path as necessary

# Directory where the mosaic images are saved
mosaic_images_dir = '../aoi_mosaic_images'  # Adjust the path as necessary

# Initialize an empty dictionary to store the images
aoi_images_dict = {}

# List all shapefiles in the directory
aoi_shapefiles = [f for f in os.listdir(AOI_SHAPEFILES_DIR) if f.endswith('.shp')]

# Extract AOI IDs from the shapefile names
aoi_ids = [os.path.splitext(f)[0] for f in aoi_shapefiles]

if DEBUG:
    aoi_ids = aoi_ids[:1]  # Limit the number of AOIs to load for debugging

# Iterate over each AOI ID and attempt to load the corresponding mosaic
for aoi_id in aoi_ids:
    mosaic_filename = f'{aoi_id}_mosaic.tif'
    mosaic_file_path = os.path.join(mosaic_images_dir, mosaic_filename)
    print(f"Processing AOI: {aoi_id}")
    
    # Check if the mosaic file exists
    if not os.path.exists(mosaic_file_path):
        raise FileNotFoundError(f"Mosaic file not found for AOI {aoi_id}: {mosaic_file_path}")
    else:
        # Open the image using rasterio
        with rasterio.open(mosaic_file_path) as src:
            # Read the image bands
            img_data = src.read()
            # Store the image data in the dictionary
            aoi_images_dict[aoi_id] = img_data
        print(f"Mosaic image loaded for AOI: {aoi_id}")

print(f"\nLoaded {len(aoi_images_dict)} AOI images into the dictionary.")


Processing AOI: aoi_2021_04
Mosaic image loaded for AOI: aoi_2021_04

Loaded 1 AOI images into the dictionary.


 ## Run Inference on Each AOI

In [8]:
import gc
from memory_profiler import profile
import cv2


# @profile
def aoi_image_to_has_strip_confidence(model: torch.nn.Module, aoi_image: np.ndarray, tiles_heightwise: int, tiles_lengthwise: int, 
                         model_input_pixels_per_side: int, model_output_tiles_per_side: int, stride: int) -> torch.Tensor:
    """
    Function to run inference on an AOI image and return the confidence map for the presence of a strip in each tile.

    Args:
        model (torch.nn.Module): The trained model used for inference.
        aoi_image (np.ndarray): The input AOI image as a NumPy array.
        tiles_heightwise (int): The number of tiles along the height of the image.
        tiles_lengthwise (int): The number of tiles along the length of the image.
        model_input_pixels_per_side (int): The number of pixels per side of the model's input.
        model_output_tiles_per_side (int): The number of tiles per side of the model's output.
        stride (int): The stride value for sliding the window over the image.

    Returns:
        torch.Tensor: A tensor representing the confidence map for the presence of a strip in each tile.
    """
    
    model.eval()

    # Resample aoi_image so that it has dims (tiles_heightwise * model_input_pixels_per_side / model_output_tiles_per_side, tiles_lengthwise * model_input_pixels_per_side / model_output_tiles_per_side)
    if not model_input_pixels_per_side % model_output_tiles_per_side == 0:
        raise ValueError("model_input_pixels_per_side must be divisible by model_output_tiles_per_side.")
    pixels_per_tile = model_input_pixels_per_side // model_output_tiles_per_side
    pixels_height = tiles_heightwise * pixels_per_tile
    pixels_width = tiles_lengthwise * pixels_per_tile

    image_t = np.transpose(aoi_image, axes=(1, 2, 0))
    aoi_image_resampled_t = cv2.resize(image_t, (pixels_width, pixels_height), interpolation=cv2.INTER_CUBIC)
    aoi_image_resampled = np.transpose(aoi_image_resampled_t, axes=(2, 0, 1))
    aoi_tensor = torch.from_numpy(aoi_image_resampled).unsqueeze(0).float()

    has_strip = torch.zeros(tiles_heightwise, tiles_lengthwise)
    
    end_idx_list = list(range(model_input_pixels_per_side, tiles_heightwise, stride)) + [tiles_heightwise]
    end_jdx_list = list(range(model_input_pixels_per_side, tiles_lengthwise, stride)) + [tiles_lengthwise]
    
    for end_idx in tqdm(end_idx_list, desc=f"Scanning AOI Rowwise", leave=False):
        for end_jdx in end_jdx_list:
            start_idx = end_idx - model_input_pixels_per_side
            start_jdx = end_jdx - model_input_pixels_per_side

            input_tensor = aoi_tensor[:, :, start_idx * pixels_per_tile:end_idx * pixels_per_tile, start_jdx * pixels_per_tile:end_jdx * pixels_per_tile]

            with torch.no_grad():
                output_tensor = model(input_tensor).squeeze()

            has_strip[start_idx:end_idx, start_jdx:end_jdx] = torch.max(
                output_tensor, 
                has_strip[start_idx:end_idx, start_jdx:end_jdx]
            )
            gc.collect()

    return has_strip

In [10]:
aoi_confidence_maps = {}

for aoi_name, aoi_image in tqdm(aoi_images_dict.items(), desc="Processing AOIs"):
    # Areas 2021_01 and 2021_02 have 1525 tiles heightwise
    tiles_lengthwise = 1541
    if aoi_name in ['aoi_2021_01', 'aoi_2021_02']:
        tiles_heightwise = 1525
    elif aoi_name in [
            'aoi_2021_04',
            'aoi_2022_01',
            'aoi_2024_01',
            'aoi_2020_02',
            'aoi_2020_03',
            'aoi_2020_01',
            'aoi_2022_02',
            'aoi_2023_01',
            'aoi_2021_03',
    ]:
        tiles_heightwise = 1527
    else:
        tiles_heightwise = None
        raise ValueError(f"Unexpected AOI name: {aoi_name}")

    print(f"Processing AOI: {aoi_name}")
    confidence_map = aoi_image_to_has_strip_confidence(
        model=model,
        aoi_image=aoi_image,
        tiles_heightwise=tiles_heightwise,
        tiles_lengthwise=tiles_lengthwise,
        model_input_pixels_per_side=train_run.config['resolution'],
        model_output_tiles_per_side=train_run.config['resolution'],
        stride=INPUT_IMAGE_SIDE_LEN_PX // 2
    )
    aoi_confidence_maps[aoi_name] = confidence_map
    gc.collect()

Processing AOIs:   0%|          | 0/1 [00:00<?, ?it/s]

Processing AOI: aoi_2021_04


Scanning AOI Rowwise:   0%|          | 0/13 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
has_strip_maps = {}
for aoi_name, confidence_map in aoi_confidence_maps.items():
    has_strip_map = confidence_map > THRESHOLD
    if ADD_BUFFER:
        has_strip_map = add_buffer_to_label(has_strip_map, num_buffer_tiles=20, buffer_type=BUFFER_TYPE)
    has_strip_maps[aoi_name] = has_strip_map

## Make submission CSV

In [ ]:
submission_df = has_strip_tensors_to_submission_csv(has_strip_maps)

## Visualize one AOI confidence map and the corresponding image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from skimage.transform import resize
import matplotlib.colors as mcolors
import torch

# Function to normalize and prepare the satellite image for display
def prepare_satellite_image(image_data):
    # Stack bands into RGB format
    img_rgb = np.dstack((image_data[0], image_data[1], image_data[2]))
    # Normalize the image for display
    img_rgb = img_rgb.astype(float)
    img_rgb = (img_rgb - img_rgb.min()) / (img_rgb.max() - img_rgb.min())
    return img_rgb

# Function to overlay the confidence map on the satellite image
def overlay_confidence_map(satellite_image, confidence_map):
    print("Overlaying confidence map...")
    # Convert confidence_map to NumPy array if it's a PyTorch tensor
    if isinstance(confidence_map, torch.Tensor):
        confidence_map = confidence_map.detach().cpu().numpy()
    
    plt.figure(figsize=(12, 12))
    plt.imshow(satellite_image)
    plt.title("Satellite Image with Confidence Map Overlay")
    plt.axis('off')
    
    # Resize the confidence map to match the satellite image dimensions
    confidence_map_resized = resize(confidence_map, (satellite_image.shape[0], satellite_image.shape[1]),
                                    order=1, preserve_range=True, anti_aliasing=False)
    
    # Create a color map for the confidence map
    cmap = plt.cm.Reds
    cmap.set_under(color='none')  # Make values below the threshold transparent
    
    # Define a threshold for visualization
    THRESHOLD = 0.5  # Adjust based on your data
    
    # Overlay the confidence map
    plt.imshow(confidence_map_resized, cmap=cmap, alpha=0.5, vmin=THRESHOLD, vmax=1)
    
    # Add a colorbar
    plt.colorbar(label='Confidence Score')
    
    plt.show()
    print("Confidence map plot displayed.")

# Function to overlay the has-strip map on the satellite image
def overlay_has_strip_map(satellite_image, has_strip_map):
    print("Overlaying has-strip map...")
    # Convert has_strip_map to NumPy array if it's a PyTorch tensor
    if isinstance(has_strip_map, torch.Tensor):
        has_strip_map = has_strip_map.detach().cpu().numpy()
    
    plt.figure(figsize=(12, 12))
    plt.imshow(satellite_image)
    plt.title("Satellite Image with Has-Strip Map Overlay")
    plt.axis('off')
    
    # Resize the has-strip map to match the satellite image dimensions
    has_strip_map_resized = resize(has_strip_map.astype(float), (satellite_image.shape[0], satellite_image.shape[1]),
                                   order=0, preserve_range=True, anti_aliasing=False)
    
    # Create a colormap for the has-strip map
    cmap = mcolors.ListedColormap(['none', 'red'])
    bounds = [0, 0.5, 1]
    norm = mcolors.BoundaryNorm(bounds, cmap.N)
    
    # Overlay the has-strip map
    plt.imshow(has_strip_map_resized, cmap=cmap, norm=norm, alpha=0.5)
    
    # Add a legend
    legend_elements = [
        Patch(facecolor='red', edgecolor='red', label='Has-Strip Area')
    ]
    plt.legend(handles=legend_elements, loc='lower right')
    
    plt.show()
    print("Has-strip map plot displayed.")

# Example usage
# Assuming you have 'aoi_image', 'confidence_map', and 'has_strip_map' variables
satellite_image = prepare_satellite_image(aoi_image)
overlay_confidence_map(satellite_image, confidence_map)
overlay_has_strip_map(satellite_image, has_strip_map)


 ## Notes

In [ ]:
# %%
# - Ensure that the coordinate reference systems (CRS) are consistent throughout the process.
# - Verify the indexing of rows and columns in the `tensor_to_submission_csv` function to match the competition requirements.
# - The code assumes that the helper functions are correctly defined in `inference_utils.py` and `train_utils.py`.
# - Adjust paths and constants as necessary based on your project structure.


 ## Conclusion

In [ ]:
# %%
print("Inference process completed for all AOIs. Submission files are ready.")

